# Atividade 03 - Parte 01

## Modelagem de Dispersão

- Implementar uma função para determinar a classe de estabilidade de Pasquil para diferentes condições atmosféricas.

- Implementar a função de estimativa de coefiente de dispersão (sigmaYZ) para todas as classes de estabilidade.

- Implementar a função de estimativa de sobrelevação da pluma utilizando os métodos de Davidson-Bryant, Holland e Briggs (quem fizer o método de Briggs ganha um ponto a mais). Deve ser considerado o efeito Tip-Downwash

- Implementar a função do modelo gaussiano

- Realizar simulações com o script criado, utilizando diferentes classes de estabilidade, velocidades do vento, alturas de chaminé. Considere a taxa de emissão que você estimou na primeira atividade. Encontre a altura de chaminé necessária para que as concentrações não violem os padrões da Resolução CONAMA 491.

- Faça figuras e discuta os resultados.

## Implementar função para determinar classes de estabilidade de Pasquill 

Utilizando o EZ-AERMOD foram importados dados de diferentes condições atmosférias e classificando conforme estação do ano. Posteriormente foi implementada a classificação de estabilidade de Pasqui conforme faixas de valores forncidadas pela universidade Federal do Espírito Santo.


In [12]:
import os
import pandas as pd

# Função para determinar a classe de estabilidade
def determinar_classe_estabilidade(row):
    u = row['u (m/s)']
    FCS = row['FCS (W/m2)']
    cco = row['cco']
    
    # Verificar se FCS é numérico
    if pd.isna(FCS) or not isinstance(FCS, (int, float)):
        return 'Unknown'  # ou qualquer valor padrão desejado
    
    FCS = float(FCS)  # Converte para float se não for
    
    if u < 2:
        if FCS > 700:
            return 'A'
        elif 350 <= FCS <= 700:
            return 'A - B'
        elif FCS < 350:
            return 'B'
    elif 2 <= u < 3:
        if FCS > 700:
            return 'A - B'
        elif 350 <= FCS <= 700:
            return 'B'
        elif FCS > 350:
            return 'C'
        if cco >= 4:
            return 'E'
        elif cco <= 3:
            return 'F'
    elif 3 <= u < 5:
        if FCS > 700:
            return 'B'
        elif 350 <= FCS <= 700:
            return 'B - C'
        elif FCS < 350:
            return 'C'
        if cco >= 4:
            return 'D'
        elif cco <= 3:
            return 'E'
    elif 5 <= u <= 6:
        if FCS > 700:
            return 'C'
        elif 350 <= FCS <= 700:
            return 'C - D'
        elif FCS < 350:
            return 'D'
        if cco >= 4:
            return 'D'
        elif cco <= 3:
            return 'D'
    elif u > 6:
        if FCS > 700:
            return 'C'
        elif 350 <= FCS <= 700:
            return 'D'
        elif FCS < 350:
            return 'D'
        if cco >= 4:
            return 'D'
        elif cco <= 3:
            return 'D'

## Testando implementação para diferentes condições atmosféricas 

In [13]:
# Pegando o caminho do diretório que estou
rootPath = os.getcwd()
print(rootPath)

# Definindo o diretório com os arquivos metar
meteoSFC = os.path.join(rootPath, 'inputs', 'meteoSFC')
print(meteoSFC)

# Listando os arquivos dentro do diretório
files = os.listdir(meteoSFC)
print(files)

# Nome das colunas que desejamos importar
columns = ['year', 'month', 'day', 'hour', 'FCS (W/m2)', 'u (m/s)', 'cco']

# Abrindo cada arquivo dentro da pasta e acumulando em uma lista chamada de metSFC
metSFC = []
for file in files:
    file_path = os.path.join(meteoSFC, file)
    df = pd.read_csv(file_path, usecols=columns)
    
    # Convertendo FCS (W/m2) para numérico
    df['FCS (W/m2)'] = pd.to_numeric(df['FCS (W/m2)'], errors='coerce')
    
    metSFC.append(df)

# Convertendo metSFC para DataFrame
metSFC = pd.concat(metSFC, ignore_index=True)

# Criando a coluna datetime
metSFC['datetime'] = pd.to_datetime(metSFC[['year', 'month', 'day', 'hour']])

# Aplicando a função para classificar as estações
metSFC['estacao'] = metSFC['datetime'].apply(classify_season)

# Aplicando a função para determinar a classe de estabilidade
metSFC['classe_estabilidade'] = metSFC.apply(determinar_classe_estabilidade, axis=1)

# Exibindo o DataFrame final
print(metSFC)


C:\ENS5173_MayaraDargas\ENS5173-2024.1_MayaraDargasSousa
C:\ENS5173_MayaraDargas\ENS5173-2024.1_MayaraDargasSousa\inputs\meteoSFC
['meteo.sfc.csv']


NameError: name 'classify_season' is not defined

## Gráfico 

In [10]:
def determinar_estacao(mes):
    if mes in [12, 1, 2]:  # Dezembro, Janeiro, Fevereiro -> Verão
        return 'Verão'
    elif mes in [3, 4, 5]:  # Março, Abril, Maio -> Outono
        return 'Outono'
    elif mes in [6, 7, 8]:  # Junho, Julho, Agosto -> Inverno
        return 'Inverno'
    elif mes in [9, 10, 11]:  # Setembro, Outubro, Novembro -> Primavera
        return 'Primavera'
    else:
        return 'Desconhecido'

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Supondo que 'metSFC' seja seu DataFrame com os dados

# Agrupando por estação do ano e classe de estabilidade
estacoes_classes = metSFC.groupby(['estacao', 'classe_estabilidade']).size().reset_index(name='count')

# Reordenando as classes de estabilidade para exibição no gráfico
classes_ordem = ['A', 'A - B', 'B', 'B - C', 'C', 'C - D', 'D', 'E', 'F']
estacoes_classes['classe_estabilidade'] = pd.Categorical(estacoes_classes['classe_estabilidade'], categories=classes_ordem, ordered=True)

# Criando os gráficos de pizza
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))

# Iterando sobre cada estação do ano
for ax, (estacao, data) in zip(axes.flatten(), estacoes_classes.groupby('estacao')):
    labels = data['classe_estabilidade']
    sizes = data['count']
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    ax.set_title(estacao)

# Ajustando a posição da legenda
legend_text = {
    'A': 'Extremamente instável',
    'B': 'Moderadamente instável',
    'C': 'Fracamente instável',
    'D': 'Neutra',
    'E': 'Fracamente estável',
    'F': 'Moderadamente estável'
}

# Exibindo as informações da legenda
plt.figtext(1.1, 0.5, 'Legenda:', ha='center', fontsize=14, va='center')
for i, (key, value) in enumerate(legend_text.items()):
    plt.figtext(1.1, 0.45 - i*0.05, f'{key}: {value}', ha='left', fontsize=12)

# Ajustes de layout
plt.suptitle('Distribuição das Classes de Estabilidade por Estação do Ano', fontsize=16)
plt.tight_layout(rect=[0, 0.05, 0.9, 0.95])  # Ajuste do espaço para a legenda
plt.subplots_adjust(right=0.85)  # Ajuste para acomodar a legenda
plt.show()


KeyError: 'estacao'

## Implementar a função de estimativa de coefiente de dispersão (sigmaYZ) para todas as classes de estabilidade

In [16]:
import pandas as pd

# Função para calcular sigmaY e sigmaZ com base nos parâmetros
def sigmaYZ(x, classe, urbOrRural):
    if urbOrRural == 'urbano':
        if classe == 'A-B':
            sigmaY = 0.32 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.24 * x * (1 + 0.001 * x) ** 0.5
        elif classe == 'C':
            sigmaY = 0.22 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'D':
            sigmaY = 0.16 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.14 * x * (1 + 0.0003 * x) ** (-0.5)
        elif classe == 'E-F':
            sigmaY = 0.11 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0015 * x) ** (-0.5)
        else:
            raise ValueError('Classe de estabilidade errada')
    elif urbOrRural == 'rural':
        if classe == 'A':
            sigmaY = 0.22 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'B':
            sigmaY = 0.16 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.12 * x
        elif classe == 'C':
            sigmaY = 0.11 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0002 * x) ** (-0.5)
        elif classe == 'D':
            sigmaY = 0.08 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.06 * x * (1 + 0.0003 * x) ** (-0.5)
        elif classe == 'E':
            sigmaY = 0.06 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.03 * x * (1 + 0.0003 * x) ** (-1)
        elif classe == 'F':
            sigmaY = 0.04 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.016 * x * (1 + 0.0003 * x) ** (-1)
        else:
            raise ValueError('Classe de estabilidade errada')
    else:
        raise ValueError('Tipo de ambiente deve ser "urbano" ou "rural"')
    
    return sigmaY, sigmaZ

## Testando implementação para todas as classes de estabilidade

In [17]:

# Valores de x para calcular
x_values = [100, 1000, 10000]
environments = ['urbano', 'rural']
classes_urbano = ['A-B', 'C', 'D', 'E-F']
classes_rural = ['A', 'B', 'C', 'D', 'E', 'F']

# Lista para armazenar os resultados
results = []

# Calculando sigmaY e sigmaZ para cada combinação de x, classe e ambiente
for x in x_values:
    for environment in environments:
        if environment == 'urbano':
            classes = classes_urbano
        elif environment == 'rural':
            classes = classes_rural
        
        for classe in classes:
            sigmaY, sigmaZ = sigmaYZ(x, classe, environment)
            results.append({'x': x, 'classe': classe, 'environment': environment, 'sigmaY': sigmaY, 'sigmaZ': sigmaZ})

# Convertendo os resultados para um DataFrame
df_results = pd.DataFrame(results)

# Exibindo a tabela com os resultados
print(df_results)


        x classe environment       sigmaY       sigmaZ
0     100    A-B      urbano    31.378582    25.171412
1     100      C      urbano    21.572775    20.000000
2     100      D      urbano    15.689291    13.794610
3     100    E-F      urbano    10.786387     7.460038
4     100      A       rural    21.890818    20.000000
5     100      B       rural    15.920595    12.000000
6     100      C       rural    10.945409     7.921180
7     100      D       rural     7.960298     5.911976
8     100      E       rural     5.970223     2.912621
9     100      F       rural     3.980149     1.553398
10   1000    A-B      urbano   270.449362   339.411255
11   1000      C      urbano   185.933936   200.000000
12   1000      D      urbano   135.224681   122.788123
13   1000    E-F      urbano    92.966968    50.596443
14   1000      A       rural   209.761770   200.000000
15   1000      B       rural   152.554014   120.000000
16   1000      C       rural   104.880885    73.029674
17   1000 

## Implementar a função de estimativa de sobrelevação da pluma

## Efeito Tip-Downwash

In [45]:
#determinar a tendência da pluma a ser dirigida em direção ao solo junto a chaminédef Holland(vs, d, u, p, Ts, Tamb):
def hg_tip_downwash(hg, d, vs, u):
    hg2 = hg + (2*d * ((vs/u)-1.5))
    return hg2

### Método de Davidson-Bryant

In [46]:
# Função para estimar o deltaH com base na equação de Davidson-Bryant

def DavidsonBryant(d, vs, u, Ts, Tamb):
    deltaH_DavidsonBryant = (d*(vs/u)**(1.4))*(1+(Ts-Tamb)/Ts)
    return deltaH_DavidsonBryant

# d: diâmetro da chaminé (m)
# vs: velocidade do efluente na saída da chaminé (m/s)
# u: velocidade do vento a 10 metros (m/s)
# deltaT: temperatura do gás na chaminé menos a tempeatura ambiente (K)
# Ts: Tempetarura do gás na saída da chaminé (K)
# Tamb: Temperatura ambiente

### Método de Holland 

In [47]:
# Função para estimar o deltaH com base na equação de Holland

def Holland(vs, d, u, p, Ts, Tamb, classe):
    deltaH_Holland = (vs * d / u) * (1.5 + (2.68 * 10**-3) * (p * ((Ts - Tamb) / Ts) * d))

    if classe == 'A':
        deltaH_Holland *= 1.20  # aumenta 20%
    elif classe == 'B':
        deltaH_Holland *= 1.15  # aumenta 15%
    elif classe == 'C':
        deltaH_Holland *= 1.10  # aumenta 10%
    elif classe == 'E':
        deltaH_Holland *= 0.90  # diminui 10%
    elif classe == 'F':
        deltaH_Holland *= 0.80  # diminui 20%
    
    return deltaH_Holland

# Se condições atmosféricas instáveis fazer acréscimo de 10-20% de delta H em delta H (se classe igual a A acrescentar 20%, se classe iguaal a B acrestar 15% e se classe igual a C acrescentar 10%)
# Se forem estáveis, diminui-se igual quantidade (Se classe igual a E diminui 10% e se classe igual a F diminui 20%)

### Método de Briggs 

In [48]:
import math

# Constante pi
pi = math.pi  

# Função para calcular a vazão volumétrica (Qo)
def vazao_volumetrica(rc, vs):
    Qo = pi * (rc**2) * vs
    return Qo

# Função para calcular o parâmetro de flutuabilidade (Fb)
def flutuabilidade(Qo, Tamb, Ts):
    Fb = (9.8 / pi) * Qo * (1 - (Tamb / Ts))
    return Fb

# Função para calcular o gradiente de temperatura potencial
def grad_temp_potencial(G, classe):
    if classe == 'A':
        G = -0.020
    elif classe == 'B':
        G = -0.018
    elif classe == 'C':
        G = -0.016
    elif classe == 'D':
        G = -0.010
    elif classe == 'E':
        G = 0.005
    else:
        G = 0.028
    grad = G - (-0.0098)
    return grad

# Função para calcular o índice de estabilidade (S)
def indice_de_estabilidade(Tamb, d_theta, d_z):
    S = (9.8 / Tamb) * (d_theta / d_z)
    return S

# Função para calcular o fluxo momentâneo
def fluxo_momentaneo(vs, d, Tamb, Ts):
    Fm = (vs**2) * (d**2) * (Tamb / (4 * Ts))
    return Fm

# g: aceleração da gravidade (9,8 m²/s)
# deltaT: temperatura do gás na chaminé menos a tempeatura ambiente (K)
# Ts: Tempetarura do gás na saída da chaminé (K)
# Tamb: Temperatura ambiente
# rc: raio da chaminé (m)
# vs: velocidade do efluente na saída da chaminé (m/s)
# G: Valor médio do gradiente de temperatura ambiente


# Função principal de Briggs
def Briggs(Fb, vs, d, u, hg2, deltaT, deltaTc, S, classe, Ts):
    
    # Determinar se a atmosfera é instável ou estável com base na classe de Pasquill
    tipo_atmosfera = 'instavel' if classe in ['A', 'B', 'C', 'D'] else 'estavel'

    if tipo_atmosfera == 'instavel':
        if Fb < 55:
            deltaTc = 0.0297 * Ts * (vs**(1/3) / (d**(2/3)))
            if deltaTc > deltaT:
                H_Briggs = hg2 + 3 * d * (vs / u)
            else:
                H_Briggs = hg2 + (21.425 * ((Fb**(3/4)) / u))
        elif Fb >= 55:
            deltaTc = 0.00575 * Ts * (vs**(2/3) / (d**(1/3)))
            if deltaTc > deltaT:
                H_Briggs = hg2 + 3 * d * (vs / u)
            else:
                dH_Briggs = hg2 + (2.6 * ((Fb / u)**(1/3)))
    
    if tipo_atmosfera == 'estavel':
        deltaTc = 0.01958 * Ts * vs * (S**(1/2))
        if deltaT < deltaTc:
            H_Briggs = hg2 + 3 * d * (vs / u)
        else:
            H_Briggs = hg2 + (2.6 * ((Fb**(3/4)) / u))
    
    return H_Briggs


## Altura efetiva da pluma

#### Parâmetros Comuns
- vs: Velocidade do efluente na saída da chaminé (m/s)
- d: Diâmetro da chaminé (m)
- u: Velocidade do vento a 10 metros (m/s)
- Ts: Temperatura do gás na saída da chaminé (K)
- Tamb: Temperatura ambiente (K)
- classe: Classe de estabilidade atmosférica (A, B, C, D, E, F)

#### Parâmetros Específicos
- rc: Raio da chaminé (m)
- Qo: Vazão volumétrica (m³/s)
- Fb: Parâmetro de flutuabilidade
- p: Pressão atmosférica (Pa)
- deltaT: Diferença de temperatura entre o gás e o ambiente (K)
- deltaTc: Diferença crítica de temperatura
- S: Índice de estabilidade
- hg2: Altura ajustada da chaminé considerando downwash
- G: Gradiente de temperatura potencial

In [49]:
import math

# Constante pi
pi = math.pi  

# Função para calcular a vazão volumétrica (Qo)
def vazao_volumetrica(rc, vs):
    Qo = pi * (rc**2) * vs
    return Qo

# Função para calcular o parâmetro de flutuabilidade (Fb)
def flutuabilidade(Qo, Tamb, Ts):
    Fb = (9.8 / pi) * Qo * (1 - (Tamb / Ts))
    return Fb

# Função para calcular o gradiente de temperatura potencial
def grad_temp_potencial(G, classe):
    if classe == 'A':
        G = -0.020
    elif classe == 'B':
        G = -0.018
    elif classe == 'C':
        G = -0.016
    elif classe == 'D':
        G = -0.010
    elif classe == 'E':
        G = 0.005
    else:
        G = 0.028
    grad = G - (-0.0098)
    return grad

# Função para calcular o índice de estabilidade (S)
def indice_de_estabilidade(Tamb, d_theta, d_z):
    S = (9.8 / Tamb) * (d_theta / d_z)
    return S

# Função para calcular a altura ajustada da chaminé (downwash)
def hg_tip_downwash(hg, d, vs, u):
    hg2 = hg + (2*d * ((vs/u)-1.5))
    return hg2

# Função para estimar o deltaH com base na equação de Davidson-Bryant
def DavidsonBryant(d, vs, u, Ts, Tamb):
    deltaH_DavidsonBryant = (d*(vs/u)**(1.4))*(1+(Ts-Tamb)/Ts)
    return deltaH_DavidsonBryant

# Função para estimar o deltaH com base na equação de Holland
def Holland(vs, d, u, p, Ts, Tamb, classe):
    deltaH_Holland = (vs * d / u) * (1.5 + (2.68 * 10**-3 * p * ((Ts - Tamb) / Ts) * d))

    if classe == 'A':
        deltaH_Holland *= 1.20  # aumenta 20%
    elif classe == 'B':
        deltaH_Holland *= 1.15  # aumenta 15%
    elif classe == 'C':
        deltaH_Holland *= 1.10  # aumenta 10%
    elif classe == 'E':
        deltaH_Holland *= 0.90  # diminui 10%
    elif classe == 'F':
        deltaH_Holland *= 0.80  # diminui 20%
    
    return deltaH_Holland

# Função principal de Briggs
def Briggs(Fb, vs, d, u, hg2, deltaT, deltaTc, S, classe, Ts):
    
    # Determinar se a atmosfera é instável ou estável com base na classe de Pasquill
    tipo_atmosfera = 'instavel' if classe in ['A', 'B', 'C', 'D'] else 'estavel'

    if tipo_atmosfera == 'instavel':
        if Fb < 55:
            deltaTc = 0.0297 * Ts * (vs**(1/3) / (d**(2/3)))
            if deltaTc > deltaT:
                H_Briggs = hg2 + 3 * d * (vs / u)
            else:
                H_Briggs = hg2 + (21.425 * ((Fb**(3/4)) / u))
        elif Fb >= 55:
            deltaTc = 0.00575 * Ts * (vs**(2/3) / (d**(1/3)))
            if deltaTc > deltaT:
                H_Briggs = hg2 + 3 * d * (vs / u)
            else:
                H_Briggs = hg2 + (2.6 * ((Fb / u)**(1/3)))
    
    if tipo_atmosfera == 'estavel':
        deltaTc = 0.01958 * Ts * vs * (S**(1/2))
        if deltaT < deltaTc:
            H_Briggs = hg2 + 3 * d * (vs / u)
        else:
            H_Briggs = hg2 + (2.6 * ((Fb**(3/4)) / u))
    
    return H_Briggs

# Exemplo de parâmetros de entrada
rc = 2.5  # raio da chaminé em metros
vs = 10  # velocidade do efluente em m/s
u = 5  # velocidade do vento a 10 metros em m/s
p_mb = 1013.25  # pressão atmosférica em mb
Ts = 600  # temperatura do gás na saída da chaminé em K
Tamb = 300  # temperatura ambiente em K
classe = 'B'  # classe de estabilidade atmosférica

# Cálculo da vazão volumétrica (Qo)
Qo = vazao_volumetrica(rc, vs)

# Cálculo do parâmetro de flutuabilidade (Fb)
Fb = flutuabilidade(Qo, Tamb, Ts)

# Cálculo do gradiente de temperatura potencial (G)
G = 0  # valor inicial de G, será ajustado na função
grad = grad_temp_potencial(G, classe)

# Cálculo do índice de estabilidade (S)
d_theta = 10  # exemplo de diferença de temperatura potencial em K
d_z = 100  # exemplo de diferença de altura em m
S = indice_de_estabilidade(Tamb, d_theta, d_z)

# Ajuste da altura da chaminé considerando downwash
hg = 100  # altura original da chaminé em metros
hg2 = hg_tip_downwash(hg, rc*2, vs, u)  # rc*2 é o diâmetro

# Cálculo do deltaT (diferença de temperatura)
deltaT = Ts - Tamb

# Cálculo dos deltaHs
H_Briggs = Briggs(Fb, vs, rc*2, u, hg2, deltaT, 0, S, classe, Ts)
deltaH_Holland = Holland(vs, rc*2, u, p_mb, Ts, Tamb, classe)
deltaH_DavidsonBryant = DavidsonBryant(rc*2, vs, u, Ts, Tamb)

# Exibindo os resultados
print("H_Briggs (m):", H_Briggs)
print("deltaH_Holland (m):", deltaH_Holland)
print("H_DavidsonBryant:", deltaH_DavidsonBryant + hg2)

H_Briggs (m): 115.24885571203681
deltaH_Holland (m): 95.3209125
H_DavidsonBryant: 124.79261866159341


In [50]:
# Exemplo de parâmetros de entrada
rc = 2.5  # raio da chaminé em metros
vs = 10  # velocidade do efluente em m/s
u = 5  # velocidade do vento a 10 metros em m/s
p_mb = 1013.25  # pressão atmosférica em mb
Ts = 600  # temperatura do gás na saída da chaminé em K
Tamb = 300  # temperatura ambiente em K
classe = 'B'  # classe de estabilidade atmosférica

# Cálculo da vazão volumétrica (Qo)
Qo = vazao_volumetrica(rc, vs)

# Cálculo do parâmetro de flutuabilidade (Fb)
Fb = flutuabilidade(Qo, Tamb, Ts)

# Cálculo do gradiente de temperatura potencial (G)
G = 0  # valor inicial de G, será ajustado na função
grad = grad_temp_potencial(G, classe)

# Cálculo do índice de estabilidade (S)
d_theta = 10  # exemplo de diferença de temperatura potencial em K
d_z = 100  # exemplo de diferença de altura em m
S = indice_de_estabilidade(Tamb, d_theta, d_z)

# Ajuste da altura da chaminé considerando downwash
hg = 100  # altura original da chaminé em metros
hg2 = hg_tip_downwash(hg, rc*2, vs, u)  # rc*2 é o diâmetro

# Cálculo do deltaT (diferença de temperatura)
deltaT = Ts - Tamb

# Cálculo dos deltaHs
H_Briggs = Briggs(Fb, vs, rc*2, u, hg2, deltaT, 0, S, classe, Ts)
deltaH_Holland = Holland(vs, rc*2, u, p_mb, Ts, Tamb, classe)
deltaH_DavidsonBryant = DavidsonBryant(rc*2, vs, u, Ts, Tamb)

# Exibindo os resultados
print("H_Briggs (m):", H_Briggs)
print("deltaH_Holland (m):", deltaH_Holland)
print("H_DavidsonBryant:", deltaH_DavidsonBryant + hg2)

H_Briggs (m): 115.24885571203681
deltaH_Holland (m): 95.3209125
H_DavidsonBryant: 124.79261866159341


## Implementar a função do Modelo Gaussiano 

In [188]:
def modeloGaussiano(qs,sigmaY,sigmaZ,u,y,z,H):
    termo1 = qs/(2*np.pi*sigmaY*sigmaZ*u)
    termo2 = np.exp((-y**2)/(2*sigmaY**2))
    termo3 = np.exp((-(z-H)**2)/(2*sigmaZ**2)) + np.exp((-(z+H)**2)/(2*sigmaZ**2))
    conc = termo1*termo2*termo3
    conc = conc*10**6
    return conc